**Introduction**

Welcome to our notebook. The main goal of this notebook is to find the best solution for the “M5 Competition - Accuracy”. Specifically, we will try to predict the future sales for Walmart based on hierarchical sales data, generously made available by Walmart, starting at the item level and aggregating to that of departments, product categories and stores in three geographical areas of the US: California, Texas, and Wisconsin.

In this Kernel, we will explain each stage of analysis, such as cleaning and editing the existing data. Then, we will apply 3 different machine learning algorithms in order to achieve the best results in terms of accuracy. More specific, we will use Facebook Prophet, Random Forest Regression and XGBoost.

**What is Facebook Prophet?** Prophet is a procedure for forecasting time series data based on an additive model where non-linear trends are fit with yearly, weekly, and daily seasonality, plus holiday effects. It works best with time series that have strong seasonal effects and several seasons of historical data. Prophet is robust to missing data and shifts in the trend, and typically handles outliers well.

**What is Random Forest Regression** A random forest is a meta estimator that fits a number of classifying decision trees on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting.

**What is XGBoost** XGBoost is an optimized distributed gradient boosting library designed to be highly efficient, flexible and portable. It implements machine learning algorithms under the Gradient Boosting framework. XGBoost provides a parallel tree boosting (also known as GBDT, GBM) that solve many data science problems in a fast and accurate way.

**Dataset** The dataset that we will apply the machine learning models constitutes sales data that they were provided by Walmart. More specific, the M5 dataset contains sales data for 3,049 different products, classified in 3 product categories (Hobbies, Foods and Household).

**Data Sources Explanation** In order to proceed in the analysis, 3 different files are provided:

sell_prices.csv: Contains information about the price of the products sold per store and date. sales_train.csv: Contains the historical daily unit sales data per product and store. calendar: Contains information about the dates the products are sold.

Firstly, we need to import all the necessary libraries and files

In [ ]:
from __future__ import print_function
from sklearn.model_selection import train_test_split
from ipywidgets import interact, interactive, fixed, interact_manual, Dropdown
from ipywidgets import Layout
import ipywidgets as widgets
from IPython.display import display
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import xgboost as xgb
from xgboost import XGBRegressor
from fbprophet import Prophet
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In order to understand the problem we need to read the data from each data source. Thus, we will have a better view of the data that will help us to schedule our actions properly.

In [ ]:
#DATASET WITH DATES
calendar = pd.read_csv("../input/m5-forecasting-accuracy/calendar.csv")

#DATASET WITH SALES
sales_eval = pd.read_csv("../input/m5-forecasting-accuracy/sales_train_evaluation.csv")

#DATASET WITH THE NEXT 28 DAILY SALES
sales = pd.read_csv("../input/m5-forecasting-accuracy/sales_train_validation.csv")




In [ ]:
#CALENDAR HEAD
print(calendar.head())


In [ ]:
#SALES HEAD
print(sales_eval.head())

It is important to eliminate duplicate values. So we need to aggregate all the products in order to create a file with unique products.

In [ ]:
#CREATE FILE WITH ALL PRODUCTS REGARDLESS OF THE ANNEX SO THAT WE DONT HAVE DUPLICATE PRODUCTS
sales_final=sales.groupby('item_id',as_index=False).sum()
sales_final['total']=sales_final.iloc[:,1:].sum(axis=1)

#SORT THE FINAL FILE IN DESCENDING ORDER TO APPEAR IN THE DROP DOWN MENU BASED ON SALES 
sales_total_sort=sales_final.iloc[:,[0,-1]].sort_values(by='total',ascending=False)

#GROUP BY THE FILE WITH THE REAL 28 NEXT DAILY SALES
sales_eval_final=sales_eval.groupby('item_id',as_index=False).sum()

print(sales_eval_final.head())

In addition, we will split the file that we created based on the product categories.

In [ ]:
#SPLIT  DATASET BASED ON THE CATEGORIES OF THE PRODUCTS &
#SORT THEM IN DESCENDING ORDER TO APPEAR IN THE DROP DOWN MENU BASED ON SALES

#HOBBIES
hobbies = sales_final[sales_final['item_id'].str.split('_').str[0]=='HOBBIES']
hobbies.shape

hobbies_sort=hobbies.iloc[:,[0,-1]].sort_values(by='total',ascending=False)
print(hobbies_sort.head())

#HOUSEHOLD
household = sales_final[sales_final['item_id'].str.split('_').str[0]=='HOUSEHOLD']
household.shape

household_sort=household.iloc[:,[0,-1]].sort_values(by='total',ascending=False)
print(household_sort.head())

#FOODS
foods = sales_final[sales_final['item_id'].str.split('_').str[0]=='FOODS']
foods.shape

foods_sort=foods.iloc[:,[0,-1]].sort_values(by='total',ascending=False)
print(foods_sort.head())

We created a few functions in order to better manage the data. Thus, it will be easier to build the models properly without any mistakes. So, as we can easily understand, this procedure is really vital.

First we created a function for the date features, such as date, month, year. Thus, it is easier to call each of them when it is neccessary. Actually, we created a variable X which containts the dataframe with the date features.

In [ ]:
#CREATE A FUNCTION TO ADD FEATURES ON TRAIN SET BASED ON DATES
def date_features(df, label=None,r=None):
    df = df.copy()

    df['date'] = df.Date
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['dayofweek'] = [i for i in calendar['wday'][:r]]
    df['quarter'] = df['date'].dt.quarter
    df['dayofyear'] = df['date'].dt.dayofyear
    df['dayofmonth'] = df['date'].dt.day
    df['weekofyear'] = df['date'].dt.isocalendar().week.astype('int64')
    df['Event_1']=np.where(calendar['event_type_1'][:r].isna(),0,1)
    df['Event_2']=np.where(calendar['event_type_2'][:r].isna(),0,1)
    #SATURDAY OR SUNDAY
    df['S&S']=[1 if calendar.iloc[i,3]==1 else 1 if calendar.iloc[i,3]==2 else 0 for i in range(len(calendar[:r]))]
    
    X = df[['dayofweek','quarter','month','year',
           'dayofyear','dayofmonth','weekofyear','Event_1','Event_2','S&S']]
    if label:
        y = df[label]
        return X, y
    return X

In [ ]:
#CREATE A FUNCTION TO ADD FEATURES ON TEST SET BASED ON DATES
def date_features_next28(df, label=None,r1=None,r2=None):
    df = df.copy()

    df['date'] = df.Date
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['dayofweek'] = [i for i in calendar['wday'][r1:r2]] 
    df['quarter'] = df['date'].dt.quarter
    df['dayofyear'] = df['date'].dt.dayofyear
    df['dayofmonth'] = df['date'].dt.day
    df['weekofyear'] = df['date'].dt.isocalendar().week.astype('int64')
    df['Event_1']=np.where(calendar['event_type_1'][r1:r2].isna(),0,1)
    df['Event_2']=np.where(calendar['event_type_2'][r1:r2].isna(),0,1)
    #SATURDAY OR SUNDAY
    df['S&S']=[1 if calendar.iloc[i,3]==1 else 1 if calendar.iloc[i,3]==2 else 0 for i in range(len(calendar[r1:r2]))]
    
    X = df[['dayofweek','quarter','month','year',
           'dayofyear','dayofmonth','weekofyear','Event_1','Event_2','S&S']]
    if label:
        y = df[label]
        return X, y
    return X

We need to mention that this Kernel is dynamic. Specifically, we created a drop down menu in order to give users the capability to choose between various reports/results. To be more specific, the choices are: XGBoost, FB Prophet, RandomForest, Plot Times Series, See Trends, XGBoost2

In [ ]:
#CREATE THE DROP DOWN WITH THE PRODUCTS CATEGORIES, RANGE OF SALES, THE PRODUCT, VISUALIZATIONS AND PREDICTIONS ALGORITHMS

#CHOICES OF DROP DOWN MENU

cat={'HOBBIES':hobbies_sort,'HOUSEHOLD':household_sort,'FOODS':foods_sort,'TOTAL':sales_total_sort}
opt=sales_total_sort.sort_values('total')['total'][:]
AlgoChoices={3:"Select",4:"Plot Time Series",5:"See Trends",6:"Plot Total Sales",0:"FB Prophet",1:"XGBoost",2:"XGBoost2",7:"RandomForest"}

catW=widgets.Dropdown(options = cat.keys(),description='Category:')

#RANGE OF SALES
range_slider=widgets.SelectionRangeSlider(options=opt,index=(200,2000),layout=Layout(width='50%'),step=100,description='Sales Number',disabled=False, feature_weights=True)

idW=widgets.Dropdown(description='Product:')

algoChoice=widgets.Dropdown(options=AlgoChoices.values(),description='Algorithm:')

Below you will find the function which is important in order to call the option "Plot Time Series".

In [ ]:
#DAILY SALES FOR EACH PRODUCT
def Plot_TimeSeries():
    
    df_sales_temp = sales_final[sales_final['item_id']==idW.value].iloc[:,1:-1].sum(axis=0)
    df_sales_temp.index = range(1913)
    
    Sales_dict ={'Sales' : df_sales_temp,'Date' : pd.to_datetime(calendar['date'][:-56])}
   
    product_sales_per_date = pd.DataFrame(Sales_dict)
    
    #DAILY SALES PLOT
    sns.set(font_scale=2)  # big
    fig, ax = plt.subplots(figsize=(40,14))
    a = sns.lineplot(x="Date", y="Sales", data=product_sales_per_date)
    a.set_title("Daily Sales Data",fontsize=30)
    plt.show()

Below you will find the function which is important in order to call the option "See Trends".

In [ ]:
#TRENDS
def See_Trends():
    
    df_sales_temp = sales_final[sales_final['item_id']==idW.value].iloc[:,1:-1].sum(axis=0)
    df_sales_temp.index = range(1913)   
    
    Sales_dict ={'Sales' : df_sales_temp,'Date' : pd.to_datetime(calendar['date'][:-56])}
    
    product_sales_per_date = pd.DataFrame(Sales_dict)
    
    #Call the Function
    X, y = date_features(product_sales_per_date, label='Sales')
    df_new = pd.concat([X, y], axis=1)
    df_new.head()

    #Plotting the Features to see trends (SALES PER MONTH)
    sns.set(font_scale=2)  # big
    fig, ax = plt.subplots(figsize=(28,10))
    palette = sns.color_palette("mako_r", 4)
    a = sns.barplot(x="month", y="Sales",hue = 'year',data=df_new)
    a.set_title("Store Sales Data",fontsize=15)
    plt.legend(loc='best')
    plt.show()

Below you will find the function which is important in order to call "Plot Total Sales".

In [ ]:
#PLOT FOR TOTAL SALES
def Plot_Total_Sales():
    
    df_sales_temp = sales_final[sales_final['item_id']==idW.value].iloc[:,1:-1].sum(axis=0)
    df_sales_temp.index = range(1913)   
    
    Sales_dict ={'Sales' : df_sales_temp,'Date' : pd.to_datetime(calendar['date'][:-56])}
    
    product_sales_per_date = pd.DataFrame(Sales_dict)   
    
    #Call the Function
    X, y = date_features(product_sales_per_date, label='Sales')
    df_new = pd.concat([X, y], axis=1)
    df_new.head()
    
    fig,(ax1,ax2,ax3,ax4)= plt.subplots(nrows=4)
    fig.set_size_inches(30,45)
    
    monthAggregated = pd.DataFrame(df_new.groupby("month")["Sales"].sum()).reset_index().sort_values('Sales')
    sns.barplot(data=monthAggregated,x="month",y="Sales",ax=ax1)
    ax1.set(xlabel='Month', ylabel='Total Sales received')
    ax1.set_title("Total Sales received By Month",fontsize=15)

    monthAggregated = pd.DataFrame(df_new.groupby("dayofweek")["Sales"].sum()).reset_index().sort_values('Sales')
    sns.barplot(data=monthAggregated,x="dayofweek",y="Sales",ax=ax2)
    ax2.set(xlabel='dayofweek', ylabel='Total Sales received')
    ax2.set_title("Total Sales received By Weekday",fontsize=15)

    monthAggregated = pd.DataFrame(df_new.groupby("quarter")["Sales"].sum()).reset_index().sort_values('Sales')
    sns.barplot(data=monthAggregated,x="quarter",y="Sales",ax=ax3)
    ax3.set(xlabel='Quarter', ylabel='Total Sales received')
    ax3.set_title("Total Sales received By Quarter",fontsize=15)

    monthAggregated = pd.DataFrame(df_new.groupby("year")["Sales"].sum()).reset_index().sort_values('Sales')
    sns.barplot(data=monthAggregated,x="year",y="Sales",ax=ax4)
    ax4.set(xlabel='year', ylabel='Total Sales received')
    ax4.set_title("Total Sales received By year",fontsize=15)

Below you will find the function which is vital for FB Prophet's implementation

In [ ]:
#FUNCTION FOR FBPROPHET 
def my_Prophet():
    
    df_sales_temp = sales_final[sales_final['item_id']==idW.value].iloc[:,1:-1].sum(axis=0)
    df_sales_temp.index = range(1913)       
    
    Sales_dict ={'y' : df_sales_temp,'ds' : pd.to_datetime(calendar['date'][:-56])}
    
    product_sales_per_date = pd.DataFrame(Sales_dict)
    
    #SPLIT THE DATA SET
    split_date = '2016-02-01'
    subset1 = (product_sales_per_date['ds'] <= split_date)
    subset2 = (product_sales_per_date['ds'] > split_date)


    X_tr = product_sales_per_date.loc[subset1]
    X_tst = product_sales_per_date.loc[subset2]
    print("train shape",X_tr.shape)
    print("test shape",X_tst.shape)

    pd.plotting.register_matplotlib_converters()
    f, ax = plt.subplots(figsize=(42,15))
    X_tr.plot(kind='line', x='ds', y='y', color='blue', label='Train', ax=ax)
    X_tst.plot(kind='line', x='ds', y='y', color='red', label='Test', ax=ax)
    plt.title('Sales Amount Traning and Test data')
    plt.show()
    model =Prophet()
    model.add_country_holidays(country_name='US')
    model.add_seasonality(name='custom_seasonality', period=28,fourier_order=5)
    model.fit(X_tr)

    
    future_dates = pd.DataFrame({'ds':pd.date_range('2016-04-24', periods=28)})
    forecast = model.predict(future_dates)
    forecast[['ds', 'yhat']]
    
    # Plot the components of the model
    fig = model.plot_components(forecast,figsize=(45,15))

    # Plot the forecast
    f, ax = plt.subplots(1)
    f.set_figheight(15)
    f.set_figwidth(45)
    fig = model.plot(forecast,ax=ax)
    plt.show()


    # Plot the forecast with the actuals
    f, ax = plt.subplots(1)
    f.set_figheight(15)
    f.set_figwidth(45)
    ax.scatter(X_tst.ds, X_tst['y'], color='r')
    fig = model.plot(forecast, ax=ax)

    f, ax = plt.subplots(figsize=(14,5))
    f.set_figheight(15)
    f.set_figwidth(45)
    X_tst.plot(kind='line',x='ds', y='y', color='red', label='Test', ax=ax)
    forecast.plot(kind='line',x='ds',y='yhat', color='green',label='Forecast', ax=ax)
    plt.title('Forecast vs Actuals')
    plt.show()
    
    real_sales=sales_eval_final[sales_eval_final['item_id']==idW.value].iloc[:,1914:].sum(axis=0).values
    rmse=np.sqrt(mean_squared_error(real_sales, forecast['yhat']))

    print(forecast[['ds', 'yhat']])
    print("RMSE",rmse)
    

The differentiate of the dataset between train and test its a very common step in order to apply the ML models properly.

In [ ]:
def train_test_split(data, n_test):
    
    return data[:-n_test, :], data[-n_test:, :]

We will apply XGBoost in order to forecast the next 28 days for a certain products based only on 
previous historical sales. In order to apply this method properly is needed to tranform time series data to supervised. Then, we will use a function called "walk_forward_validation" which is responsible for the estimation of RMSE and MAE.

In [ ]:
# forecast monthly sales with xgboost

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[0]
    df = pd.DataFrame(data)
    cols = list()
    
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
    
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        # put it all together
    agg = pd.concat(cols, axis=1)
    
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
        
    return agg.values

In [ ]:
def xgboost_forecast(train, testX):
    
    # transform list into array
    train = np.asarray(train)
    
    # split into input and output columns
    trainX, trainy = train[:, :-1], train[:, -1]
    
    # fit model
    model= XGBRegressor(n_estimators=125,learning_rate=0.01,gamma=0.01,reg_lambda=0.01,max_depth=7)
    model.fit(trainX, trainy)
    # make a one-step prediction
    yhat = model.predict(np.asarray([testX]))
    
    return yhat[0]

In [ ]:
# walk-forward validation for univariate data
def walk_forward_validation(data, n_test):
    
    predictions = list()
    
    # split dataset
    train, test = train_test_split(data, n_test)
    
    # seed history with training dataset
    history = [x for x in train]
    
    # step over each time-step in the test set
    for i in range(len(test)):
        
        # split test row into input and output columns
        testX, testy = test[i, :-1], test[i, -1]
   
        # fit model on history and make a prediction
        yhat = xgboost_forecast(history, testX)
         # store forecast in list of predictions
        predictions.append(yhat)
        
        # add actual observation to history for the next loop
        history.append(test[i])
        
        # summarize progress
        print('>expected=%.1f, predicted=%.1f' % (testy, yhat))
    
    # estimate prediction error
    mae=mean_absolute_error(test[:, -1], predictions)
    rmse=np.sqrt(mean_squared_error(test[:, -1], predictions))
    
    return mae,rmse, test[:, -1], predictions

In [ ]:
def XGBoost():
    
    df_sales_temp = sales_final[sales_final['item_id']==idW.value].iloc[:,1:-1].sum(axis=0)
    df_sales_temp.index = range(1913)
    
    Sales_dict ={'Sales' : df_sales_temp,'Date' : pd.to_datetime(calendar['date'][:-28])}
    
    product_sales_per_date = pd.DataFrame(Sales_dict)
    
    X,y=date_features(product_sales_per_date, label='Sales',r=1941)
    
    X=X.set_index(product_sales_per_date['Date'])
    series=X
    values=y
    data = series_to_supervised(values, n_in=6, n_out=1)
    mae,rmse ,y, yhat = walk_forward_validation(data,28)
    
    return ("MAE",round(mae,4)), ("RMSE",rmse)
    

We will apply XGBoost (XGBoost2) for a second time. However, we will use some extra features such as Year or Dayofweek.

In [ ]:
def XGBoost2():
    
    df_sales_temp = sales_final[sales_final['item_id']==idW.value].iloc[:,1:-1].sum(axis=0)
    df_sales_temp.index = range(1913)
 
    Sales_dict ={'Sales' : df_sales_temp,'Date' : pd.to_datetime(calendar['date'][:-56])}
    
    product_sales_per_date = pd.DataFrame(Sales_dict)
    X,y=date_features(product_sales_per_date, label='Sales',r=1913)

    #split the dataset at 70-30 
    df_train=X.iloc[:-574,:]
    df_test=X.iloc[-574:,:] 

    X_train=df_train
    y_train=y[:-574]
    x_test=df_test
    y_test=y[-574:]

    # Initialize XGB and GridSearch
    xgb_reg = xgb.XGBRegressor(colsample_bytree= 0.7,subsample=0.4,reg_alpha=0.7,min_child_weight=30,n_estimators=50,learning_rate=0.1,gamma=0.7,reg_lambda=1.5,max_depth=3)
    xgb_reg.fit(X_train,y_train)
    pred_xgb=xgb_reg.predict(x_test)
    
    #NEW DATASET(NEXT 28 DAYS)
    df_test28next_xgb=pd.DataFrame()
    df_test28next_xgb['Date']=pd.to_datetime(calendar[calendar['date']>"2016-04-24"]['date'][:28])
    X_new=date_features_next28(df_test28next_xgb,r1=1913,r2=1941)
    
    #PREDICTIONS FOR NEXT 28 DAYS
    pred2=xgb_reg.predict(X_new)
    
    #ADD THE PREDICTIONS AND THE REAL SALES AT TEST DATA SET
    X_new['Predicted Sales']=abs(pred2.round(0))
    X_new['Real Sales']=sales_eval_final[sales_eval_final['item_id']==idW.value].iloc[:,1914:].sum(axis=0).values
   
    #THE TEST DATA SET WITH PREDICTIONS AND REAL SALES
    print(X_new)
    
    #RMSE
    rmse=np.sqrt(mean_squared_error(y_test, pred_xgb))
    print("RMSE",rmse)



In order to predict the sales for the next 28 days, we will use RandomForest, too

In [ ]:
def RandomForest():
    
    from sklearn.model_selection import train_test_split
    df_sales_temp = sales_final[sales_final['item_id']==idW.value].iloc[:,1:-1].sum(axis=0)
    df_sales_temp.index = range(1913)

    Sales_dict ={'Sales' : df_sales_temp,'Date' : pd.to_datetime(calendar['date'][:-56])}
    product_sales_per_date = pd.DataFrame(Sales_dict)
    
    X, y= date_features(product_sales_per_date, label='Sales',r=1913)
    
    Dataframe_RandomForest = pd.concat([X, y], axis=1)
    predictors=Dataframe_RandomForest.drop(['Sales'],axis=1)
    target=Dataframe_RandomForest['Sales']
    X_train,X_test_cv,y_train,y_test_cv=train_test_split(predictors, target, test_size=0.25,random_state=42)
    
    #Hypertuned Model
    model = RandomForestRegressor(n_estimators=100,oob_score=True,n_jobs =1,random_state=42,max_features='auto',min_samples_leaf=4)
    model.fit(X_train,y_train)
    
    #PREDICTION TO TRAIN SER
    pred=model.predict(X_test_cv)
    
    #NEW DATASET(NEXT 28 DAYS)
    df_test28next=pd.DataFrame()
    df_test28next['Date']=pd.to_datetime(calendar[calendar['date']>"2016-04-24"]['date'][:28])
    X_test_28=date_features_next28(df_test28next,r1=1913,r2=1941)
    
    #PREDICTIONS FOR NEXT 28 DAYS
    pred2=model.predict(X_test_28)
    
    #ADD THE PREDICTIONS AND THE REAL SALES AT TEST DATA SET
    X_test_28['Predicted Sales']=pred2.round(0)
    X_test_28['Real Sales']=sales_eval_final[sales_eval_final['item_id']==idW.value].iloc[:,1914:].sum(axis=0).values
    
    #TEST DATA SET
    print(X_test_28)
    
    #RMSE
    rmse=np.sqrt(mean_squared_error(y_test_cv, pred))
    print("RMSE",rmse)

We created a drop down menu in order to give the user some choices regarding ML models & plots. Below, you can see the available choices.

In [ ]:
#MENU
def Choice(A):
    if A==3:
        return ("Select")
    elif A==6:
        return Plot_Total_Sales()
    elif A==5:
        return See_Trends()
    elif A==4:
        return Plot_TimeSeries()
    elif A==0:    
        return my_Prophet()
    elif A==1:
        return XGBoost()
    elif A==2:
        return XGBoost2()
    elif A==7:
        return RandomForest()

Interact is used for the dropdown creation. Specifically, the user should first choose a product category, the sales range using a slidebar, the product that he/she wants to make the prediction and finally the ML model that he/she wants to apply or the plot that he/she wants to create.

In [ ]:
@interact(Category=catW,RS=range_slider,Product=idW, AC=algoChoice)
def print_idW(Category,RS,Product,AC):
    idW.options=cat[Category][(cat[Category]['total']>=RS[0]) & (cat[Category]['total']<=RS[1])]['item_id']
    return Choice(list(AlgoChoices.keys())[list(AlgoChoices.values()).index(AC)])